In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/TRAIN_FINISHED.parq')
test = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/TEST_FINISHED.parq')
val = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/VAL_FINISHED.parq')

In [3]:
X_train = train.loc[:, ~train.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_train = train.loc[:, 'label']
X_test = test.loc[:, ~test.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_test = test.loc[:, 'label']
X_val = val.loc[:, ~val.columns.isin(['label', "attacker_ip_enum", 'HAS_TCP?', 'HAS_UDP?', 'watcher_as_name'])]
Y_val = val.loc[:, 'label']
del train
del test

In [4]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

model = LGBMClassifier()
param_grid = {
    'n_estimators': [100],
    'max_depth': [7],
    'num_leaves': [31, 63 ],
    'learning_rate': [0.1],
    'subsample': [0.8],
    'reg_alpha': [0],
    'reg_lambda': [0.2, 0.3],
    'min_child_samples': [1],
    'class_weight': [None],
    'random_state': [1996],
    'boosting_type': ['gbdt'],  # 'gbdt' is the default boosting type for LightGBM
    'objective': ['binary'],  #'multiclass' for multiclass classification
    'metric': ['f1'],  # 'multi_logloss' for multiclass classification
    'scale_pos_weight': [3],  # Adjust if there is class imbalance
    'feature_fraction': [0.8],  # Similar to colsample_bytree
    'bagging_fraction': [0.8],  # Similar to subsample
    'bagging_freq': [1],  # Frequency for bagging

}



random_search = GridSearchCV(model,  param_grid, scoring='f1', cv=2, verbose=3, return_train_score=True, n_jobs=-1)
random_search.fit(X_train, Y_train)

best_params = random_search.best_params_

best_model = LGBMClassifier(**best_params)
best_model.fit(X_train, Y_train)

y_val_pred = best_model.predict(X_val)
f1_val = f1_score(Y_val, y_val_pred)

y_test_pred = best_model.predict(X_test)
f1_test = f1_score(Y_test, y_test_pred)

print("Best Hyperparameters:", best_params)
print(f'Validation F1 Score: {f1_val}')
print(f'Test F1 Score: {f1_test}')

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 608685, number of negative: 1521712
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.414918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if 

Best Hyperparameters: {'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'class_weight': None, 'feature_fraction': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'metric': 'f1', 'min_child_samples': 1, 'n_estimators': 100, 'num_leaves': 63, 'objective': 'binary', 'random_state': 1996, 'reg_alpha': 0, 'reg_lambda': 0.2, 'scale_pos_weight': 3, 'subsample': 0.8}
Validation F1 Score: 0.6021177049987687
Test F1 Score: 0.649756496181043


In [5]:
del X_test
del X_train
del X_val
del Y_train
del Y_test
del Y_val

In [6]:
del y_val_pred
del y_test_pred


In [7]:
del val

In [8]:
FINAL_TEST = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/COMP_TEST_FINISHED.parq')
len(FINAL_TEST)

18682297

In [9]:
FINAL_TEST.columns

Index(['port_count', 'risky_ports', 'attacker_ip_enum', 'attacker_as_name',
       'attacker_country', 'joined_countries_0', 'joined_countries_1',
       'joined_countries_2', 'joined_countries_3', 'joined_countries_4',
       'joined_countries_5', 'joined_countries_6', 'joined_countries_7',
       'joined_countries_8', 'joined_countries_9', 'joined_countries_10',
       'joined_countries_11', 'joined_countries_12', 'protocol_0',
       'protocol_1', 'protocol_2', 'protocol_3', 'crawl', 'exploit', 'scan',
       'spam', 'unknown', 'nan'],
      dtype='object')

In [10]:
nan_count_per_column = FINAL_TEST.isna().sum()
nan_count_per_column

port_count             0
risky_ports            0
attacker_ip_enum       0
attacker_as_name       0
attacker_country       0
joined_countries_0     0
joined_countries_1     0
joined_countries_2     0
joined_countries_3     0
joined_countries_4     0
joined_countries_5     0
joined_countries_6     0
joined_countries_7     0
joined_countries_8     0
joined_countries_9     0
joined_countries_10    0
joined_countries_11    0
joined_countries_12    0
protocol_0             0
protocol_1             0
protocol_2             0
protocol_3             0
crawl                  0
exploit                0
scan                   0
spam                   0
unknown                0
nan                    0
dtype: int64

In [11]:
import numpy as np

# Assuming FINAL_TEST is a pandas DataFrame
batch_size = 10000  # Choose an appropriate batch size

# Get the number of samples in the test set
num_samples = len(FINAL_TEST)

# Initialize an array to store the predictions
all_predictions = np.zeros(num_samples)

# Loop through the test set in batches
for i in range(0, num_samples, batch_size):
    # Extract the current batch
    current_batch = FINAL_TEST.iloc[i:i+batch_size, :].loc[:, ~FINAL_TEST.columns.isin(["attacker_ip_enum"])]

    # Make predictions on the current batch
    current_predictions = best_model.predict(current_batch)

    # Store the predictions in the array
    all_predictions[i:i+batch_size] = current_predictions


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM]

In [12]:
all_predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
FINAL_PREDICT = pd.Series(all_predictions, name='prediction')
FINAL_PREDICT

0           0.0
1           0.0
2           0.0
3           0.0
4           1.0
           ... 
18682292    0.0
18682293    0.0
18682294    0.0
18682295    0.0
18682296    0.0
Name: prediction, Length: 18682297, dtype: float64

In [14]:
FINAL_TEST

,port_count,risky_ports,attacker_ip_enum,attacker_as_name,attacker_country,joined_countries_0,joined_countries_1,joined_countries_2,joined_countries_3,joined_countries_4,...,protocol_0,protocol_1,protocol_2,protocol_3,crawl,exploit,scan,spam,unknown,nan
0,17,True,7696,0.490258,0.159045,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,17,True,7696,0.490258,0.159045,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,17,True,7696,0.490258,0.159045,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,17,True,7696,0.490258,0.159045,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,False,7543,0.399279,0.304671,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18682292,0,False,198081,0.000000,0.216196,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
18682293,0,False,198081,0.000000,0.216196,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
18682294,0,False,198081,0.000000,0.216196,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
18682295,0,False,198081,0.000000,0.216196,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [15]:
FINAL_PREDICT = pd.concat([FINAL_TEST['attacker_ip_enum'], FINAL_PREDICT], axis=1)

In [16]:
filtered_result_1 = FINAL_PREDICT[FINAL_PREDICT['prediction'] == 1].drop_duplicates(subset='attacker_ip_enum')
filtered_result_0 = FINAL_PREDICT[FINAL_PREDICT['prediction'] == 0].drop_duplicates(subset='attacker_ip_enum')

del FINAL_PREDICT

# Get IPs not in filtered_result_no_duplicates
filtered_result_0 = filtered_result_0[~filtered_result_0['attacker_ip_enum'].isin(filtered_result_1['attacker_ip_enum'])]



# Concatenate remaining_ips with filtered_result_no_duplicates
final_result = pd.concat([filtered_result_1, filtered_result_0])

del filtered_result_1
del filtered_result_0

final_result

,attacker_ip_enum,prediction
4,7543,1.0
144,7696,1.0
189,2083,1.0
198,9550,1.0
337,6232,1.0
...,...,...
18626746,196304,0.0
18638235,199918,0.0
18639976,192446,0.0
18662560,192056,0.0


In [17]:
final_result[final_result["attacker_ip_enum"] == "nan"]

,attacker_ip_enum,prediction


In [18]:
len(final_result)

49420

In [19]:
final_result.sort_values(by='attacker_ip_enum', inplace=True)

final_result

,attacker_ip_enum,prediction
21502,5,0.0
191,7,0.0
665517,21,0.0
72758,29,0.0
37390,33,0.0
...,...,...
17869691,199947,0.0
17977618,199949,0.0
17862033,199962,0.0
17857346,199964,0.0


In [20]:
final_result.isna().sum()


attacker_ip_enum    0
prediction          0
dtype: int64

In [21]:
final_result = final_result.rename(columns={'prediction': 'label'})

In [22]:
final_result = final_result.reset_index(drop=True)
final_result[[]]

""
0
1
2
3
4
...
49415
49416
49417
49418


In [23]:
final_result[["attacker_ip_enum", "label"]]

,attacker_ip_enum,label
0,5,0.0
1,7,0.0
2,21,0.0
3,29,0.0
4,33,0.0
...,...,...
49415,199947,0.0
49416,199949,0.0
49417,199962,0.0
49418,199964,0.0


In [24]:
final_result.reset_index()

,index,attacker_ip_enum,label
0,0,5,0.0
1,1,7,0.0
2,2,21,0.0
3,3,29,0.0
4,4,33,0.0
...,...,...,...
49415,49415,199947,0.0
49416,49416,199949,0.0
49417,49417,199962,0.0
49418,49418,199964,0.0


In [25]:
final_result.set_index("attacker_ip_enum").to_csv('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 3 : MODELS/SECOND - LGBM/FINAL_PREDICT.csv')